In [6]:
import pandas as pd
import csv
import numpy as np
from sklearn.decomposition import PCA

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score , GridSearchCV , StratifiedShuffleSplit
from sklearn.svm import SVC

In [3]:
from benchmark.mlp import MLP
from sklearn.metrics import confusion_matrix
import numpy as np
from benchmark.benchmark import plot_confusion_matrix , get_scores
import matplotlib.pyplot as plt
import pandas as pd
from benchmark.processor import AudioProcessor
from sklearn.preprocessing import StandardScaler
path='./music'

Using TensorFlow backend.


In [4]:
from benchmark.mlp import MLP
from sklearn.metrics import confusion_matrix
import numpy as np
from benchmark.benchmark import plot_confusion_matrix , get_scores
import matplotlib.pyplot as plt
import pandas as pd
from benchmark.processor import AudioProcessor
from sklearn.preprocessing import StandardScaler
path='./music'

In [9]:
#dataset='marsyas'
dataset='lpc'
#dataset='mfcc'
#dataset='derivatives'
myprocessor = AudioProcessor(dataset, path)
X_train, X_test, Y_train, Y_test = myprocessor.get_split_dataset()
X_train, X_test = myprocessor.scale_dataset(X_train, X_test)
print('init MLPs for {0} dataset'.format(dataset))
categories = myprocessor.inputs_le.inverse_transform(range(0, 25))
print('Classes :  {0}'.format(categories))

# mlp = MLP(X_train, X_test, Y_train, Y_test,selected_datasets[mlp_model], path)
# print(selected_datasets[mlp_model],selected_mlp_type[mlp_model])

# # Labeled data
# y_pred = mlp.predict_deep_model() if selected_mlp_type[mlp_model] == 'deep' else mlp.predict_wide_model()
# print(y_pred.shape)
# predictions.append(y_pred)

[INFO] Reading dataset lpc
[INFO] Reading attributes lpc
[INFO] Labels format : (179555, 25)
[INFO] Features format : (179555, 20)
[INFO] Splitting dataset lpc
[INFO] Labels format : (179555, 25)
[INFO] Features format : (143644, 20)
[INFO] Scaler : removing the mean and scaling to unit variance

/Users/jslecointre/anaconda/envs/gti770/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/jslecointre/anaconda/envs/gti770/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/jslecointre/anaconda/envs/gti770/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/jslecointre/anaconda/envs/gti770/lib/python3.6/site-packages/sklearn/utils/validation.py:590: DataConversionWarning: Data with input dtype object was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)



init MLPs for lpc dataset
Classes :  ['BIG_BAND' 'BLUES_CONTEMPORARY' 'COUNTRY_TRADITIONAL' 'DANCE'
 'ELECTRONICA' 'EXPERIMENTAL' 'FOLK_INTERNATIONAL' 'GOSPEL' 'GRUNGE_EMO'
 'HIP_HOP_RAP' 'JAZZ_CLASSIC' 'METAL_ALTERNATIVE' 'METAL_DEATH'
 'METAL_HEAVY' 'POP_CONTEMPORARY' 'POP_INDIE' 'POP_LATIN' 'PUNK' 'REGGAE'
 'RNB_SOUL' 'ROCK_ALTERNATIVE' 'ROCK_COLLEGE' 'ROCK_CONTEMPORARY'
 'ROCK_HARD' 'ROCK_NEO_PSYCHEDELIA']


In [14]:
data=X_train
print(X_train.shape)

(143644, 20)


In [20]:
grid_rbf

GridSearchCV(cv=2, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=2048, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=6,
       param_grid={'kernel': ['rbf'], 'C': [0.001, 0.1, 10], 'gamma': [0.001, 0.1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=4)

In [16]:
pca = PCA(0.9, svd_solver = 'full')
principalComponents = pca.fit_transform(data)
print(principalComponents.shape)

(143644, 8)


In [19]:
def grid_search_rbf(X_valid, Y_valid,k, jobs, cache_size):

    svc = SVC(cache_size=cache_size)
    param_grid_rbf = {'kernel': ['rbf'], 'C': [10 ** (-3), 10 ** (-1), 10], 'gamma': [10 ** (-3), 10 ** (-1), 10]}
    grid_rbf = GridSearchCV(svc, param_grid=param_grid_rbf, cv=k, n_jobs=jobs, scoring='accuracy', verbose=4)

    grid_rbf.fit(X_valid, Y_valid)

    print("RBF : The best hyperparameters are %s with a score of %0.2f" % (grid_rbf.best_params_, grid_rbf.best_score_))

    return grid_rbf

grid_rbf = grid_search_rbf(principalComponents, Y_valid, 2, jobs, cache_size)

Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  12 out of  18 | elapsed: 185.0min remaining: 92.5min
[Parallel(n_jobs=6)]: Done  18 out of  18 | elapsed: 258.3min finished


RBF : The best hyperparameters are {'C': 10, 'gamma': 0.1, 'kernel': 'rbf'} with a score of 0.19


In [18]:
jobs = 6
cache_size=2048
k=5
svc = SVC(cache_size=cache_size)

X_valid=principalComponents
Y_valid=np.argmax(Y_train, axis=1)

In [20]:
Y_valid.shape

(143644,)

In [21]:
X_valid.shape

(143644, 124)

In [ ]:
def grid_search_linear(X_valid, Y_valid,k, jobs, cache_size):

    svc = SVC(cache_size=cache_size)

    param_grid_linear = {'kernel': ['linear'], 'C': [10 ** (-3), 10 ** (-1), 1, 10],'class_weight': ['balanced'],'gamma': ['scale']}

    grid_linear = GridSearchCV(svc, param_grid=param_grid_linear, cv=k, n_jobs=jobs, scoring='accuracy', verbose=4)
    grid_linear.fit(X_valid, Y_valid)

    print("LINEAR : The best hyperparameters are %s with a score of %0.2f" % (grid_linear.best_params_, grid_linear.best_score_))
    
    return grid_linear


grid_linear = grid_search_linear(X_valid, Y_valid, k, jobs, cache_size)

In [ ]:
df_lin=pd.DataFrame(grid_linear.cv_results_['mean_fit_time'],index=[x['C'] for x  in grid_linear.cv_results_['params']],columns=['temps moyen du fit'])
df_lin.index.name='C'
df_lin

In [ ]:
def grid_search_rbf(X_valid, Y_valid,k, jobs, cache_size):

    svc = SVC(cache_size=cache_size)
    param_grid_rbf = {'kernel': ['rbf'], 'C': [10 ** (-3), 10 ** (-1), 10], 'gamma': [10 ** (-3), 10 ** (-1), 10]}
    grid_rbf = GridSearchCV(svc, param_grid=param_grid_rbf, cv=k, n_jobs=jobs, scoring='accuracy', verbose=4)

    grid_rbf.fit(X_valid, Y_valid)

    print("RBF : The best hyperparameters are %s with a score of %0.2f" % (grid_rbf.best_params_, grid_rbf.best_score_))

    return grid_rbf

grid_rbf = grid_search_rbf(X_valid, Y_valid, 2, jobs, cache_size)

Fitting 2 folds for each of 16 candidates, totalling 32 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  13 tasks      | elapsed: 649.8min


In [ ]:
gammas = [x['gamma'] for x  in grid_rbf.cv_results_['params']]
C_values=[x['C'] for x  in grid_rbf.cv_results_['params']]
fit_time = grid_rbf.cv_results_['mean_fit_time']
accuracies=grid_rbf.cv_results_['mean_test_score']

C_range=[10 ** (-3), 10 ** (-1), 1, 10]
gamma_range=[10 ** (-3), 10 ** (-1), 1, 10]

grid=grid_rbf
# We extract just the scores
scores = grid.cv_results_['mean_test_score']
scores = np.array(scores).reshape(len(C_range), len(gamma_range))

# Make a nice figure
plt.figure(figsize=(8, 6))
plt.subplots_adjust(left=0.15, right=0.95, bottom=0.15, top=0.95)
plt.imshow(scores, interpolation='nearest',cmap='Greens' )
plt.xlabel('gamma')
plt.ylabel('C')
plt.title('Accuracy  pour SVM non lineaire avec fonction noyau RBF SVM')
plt.colorbar()
plt.xticks(np.arange(len(gamma_range)), gamma_range, rotation=45)
plt.yticks(np.arange(len(C_range)), C_range)
plt.show()

df=pd.DataFrame(list(zip(gammas,accuracies,C_values)),columns=['gamma','','C'])
df.pivot_table(columns=['gamma'],index=['C'])